In [1]:
from ray.rllib.agents.ppo import PPOTrainer
from ray import tune
import gym

from ray import tune

train_agent=True
instance_path='resources/jsp_instances/standard/ft06.txt'
restore_agent= False
num_episodes = 10
#restore_path= 'training_checkpoints/checkpoints_az_jsslite/checkpoint-6'

config = {
    # Environment (RLlib understands openAI gym registered strings).
    #"env": "CartPole-v1",
    "env": "custom_jssp",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 1,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    #"horizon":1,
    "evaluation_duration":10,
    "model": {
        "fcnet_hiddens": [128, 128],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    },
    
}






In [2]:
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from src.jss_lite.jss_lite import jss_lite

def env_creator(config):
    env = jssp_light_obs_wrapper(jss_lite(instance_path=instance_path))
    return env


# use tune to register the custom environment for the ppo trainer
tune.register_env('custom_jssp',env_creator)

trainer = PPOTrainer(config=config)

2022-09-28 10:53:43,925	WARNING trainer.py:2540 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
(RolloutWorker pid=57077) 2022-09-28 10:53:54,182	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-09-28 10:53:55,660	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-09-28 10:53:55,661	WARNING trainer.py:2540 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.

In [7]:
import time
epoch_nr=0

for _ in range(60):
    epoch_nr+=1
    t=time.time()
    trainer.train()
    episode_reward = 0
    env = env_creator("config")
    done = False
    obs = env.reset()
    iterations=0
    while not done:
        action = trainer.compute_action(obs)
        #print(f"action: {action}")
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        #env.render(mode="human")
        iterations += 1
    #env.render()
    #env.close()
    print(f"{_+60} episode")
    print(f"episode reward:{episode_reward}")
    print(f"reward:{reward}")
    print(f"count iterations:{iterations}")
    #print(f"trainer trained: {epoch_nr} Epochs in {time.time()-t} seconds for new episode")

0 episode
episode reward:-23
reward:30
count iterations:102
1 episode
episode reward:11
reward:44
count iterations:80
2 episode
episode reward:6
reward:34
count iterations:74
3 episode
episode reward:23
reward:41
count iterations:64
4 episode
episode reward:24
reward:39
count iterations:53
5 episode
episode reward:35
reward:45
count iterations:47
6 episode
episode reward:44
reward:49
count iterations:47
7 episode
episode reward:17
reward:41
count iterations:63
8 episode
episode reward:28
reward:40
count iterations:51
9 episode
episode reward:33
reward:40
count iterations:45
10 episode
episode reward:39
reward:44
count iterations:46
11 episode
episode reward:40
reward:44
count iterations:44
12 episode
episode reward:41
reward:51
count iterations:48
13 episode
episode reward:42
reward:50
count iterations:47
14 episode
episode reward:36
reward:41
count iterations:44
15 episode
episode reward:38
reward:50
count iterations:50
16 episode
episode reward:44
reward:47
count iterations:40
17 epi

In [8]:
env.render()

In [4]:
episode_reward = 0
env = env_creator("config")

done = False
obs = env.reset()
iterations=0
while not done:
    action = trainer.compute_action(obs)
    #print(f"action: {action}")
    obs, reward, done, info = env.step(action)
    episode_reward += reward
    #env.render(mode="human")
    iterations += 1
env.render()
env.close()
print(episode_reward)
print(iterations)

2
83


In [5]:
print(obs)
print(env.env.current_timestep)

{'obs': array([0.    , 0.6   , 0.6   , 0.    , 0.8333, 1.    , 0.    , 0.4   ,
       0.4   , 0.    , 0.8333, 0.1   , 0.    , 0.7   , 0.7   , 0.    ,
       0.8333, 0.5   , 0.    , 0.9   , 0.9   , 0.    , 0.8333, 0.4   ,
       0.    , 0.1   , 0.1   , 0.    , 0.8333, 0.8   , 0.    , 0.1   ,
       0.1   , 0.    , 0.8333, 0.9   , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
      dtype=float32), 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)}
68


In [6]:
import numpy as np
display(env.env.production_list)

array([[(0, 1, 3, 6), (5, 3, 16, 26), (2, 3, 26, 35), (3, 1, 35, 40),
        (4, 4, 40, 43), (1, 4, 45, 55)],
       [(5, 0, 0, 3), (1, 0, 3, 11), (3, 0, 11, 16), (4, 1, 16, 19),
        (0, 2, 19, 25), (2, 4, 35, 36)],
       [(0, 0, 0, 1), (2, 0, 1, 6), (4, 0, 6, 15), (1, 1, 15, 20),
        (5, 5, 38, 39), (3, 2, 43, 48)],
       [(5, 1, 4, 7), (2, 1, 7, 11), (0, 3, 25, 32), (4, 5, 43, 44),
        (3, 3, 48, 51), (1, 5, 55, 59)],
       [(4, 2, 19, 24), (1, 2, 24, 34), (5, 4, 34, 38), (0, 5, 38, 44),
        (2, 5, 44, 51), (3, 4, 51, 59)],
       [(5, 2, 7, 16), (2, 2, 16, 24), (4, 3, 24, 28), (0, 4, 32, 35),
        (1, 3, 35, 45), (3, 5, 59, 68)]], dtype=object)